In [40]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt

In [64]:
class LayerUtil:
    def __init__(self):
        pass
    
    def conv(self, in_channels, filters, kernel_size):
        return nn.Conv2d(in_channels = in_channels, out_channels = filters, kernel_size = kernel_size)
    
    def pool(self, kernel_size = (2,2), stride = 2):
        return nn.MaxPool2d(kernel_size = kernel_size, stride = stride)
    
    def deconv(self, in_channels, filters, kernel_size):
            return nn.ConvTranspose2d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.helper = LayerUtil()
        self.layer1 = self.helper.conv(3, 64, (3,3))
        self.layer2 = self.helper.conv(64, 128, (3,3))
        self.pool1 = self.helper.pool((3,3), 2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        return self.sigmoid(self.pool1(self.layer2(self.layer1(inputs))))
    

In [67]:
model = UNet()
input_tensor = torch.rand(12, 3, 64,64)
final = model(input_tensor)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 62, 62]          1,792
├─Conv2d: 1-2                            [-1, 128, 60, 60]         73,856
├─MaxPool2d: 1-3                         [-1, 128, 29, 29]         --
├─Sigmoid: 1-4                           [-1, 128, 29, 29]         --
Total params: 75,648
Trainable params: 75,648
Non-trainable params: 0
Total mult-adds (M): 272.06
Input size (MB): 0.05
Forward/backward pass size (MB): 5.39
Params size (MB): 0.29
Estimated Total Size (MB): 5.73
